In [1]:
!pip install pandas opencv-python matplotlib scikit-learn
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf #using tensorflow because we can build NN and discover more patterns than just classification
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [3]:
dataset_path = r"C:\Users\aakaash\OneDrive\Desktop\QMUL\Project\PROJECT\archive"

In [4]:
meta_df = pd.read_csv(os.path.join(dataset_path, 'Meta.csv'))
train_df = pd.read_csv(os.path.join(dataset_path, 'Train.csv'))
test_df = pd.read_csv(os.path.join(dataset_path, 'Test.csv'))

In [5]:
#to load images amd preprocess
def load_images(df, base_path, img_size=(32, 32), min_val=40, max_val=120):
    images = []
    labels = []
    for index, row in df.iterrows():
        img_path = os.path.join(base_path, row['Path'])
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  #convertt to RGB
            img = np.clip(img, min_val, max_val)
            img = (img - min_val) / (max_val - min_val) #scale to 0-1
            images.append(img)
            labels.append(row['ClassId'])
    return np.array(images, dtype='float32'), np.array(labels, dtype='int32')

In [6]:
train_images, train_labels = load_images(train_df, dataset_path) #loading train images and labels
test_images, test_labels = load_images(test_df, dataset_path) #loading test images and labels

In [7]:
print(f"Number of training images: {train_images.shape[0]}")
print(f"Number of training labels: {train_labels.shape[0]}")
print(f"Number of test images: {test_images.shape[0]}")
print(f"Number of test labels: {test_labels.shape[0]}")

Number of training images: 39209
Number of training labels: 39209
Number of test images: 12630
Number of test labels: 12630


In [8]:
class_names = {
    0: "Speed limit (20km/h)",
    1: "Speed limit (30km/h)",
    2: "Speed limit (50km/h)",
    3: "Speed limit (60km/h)",
    4: "Speed limit (70km/h)",
    5: "Speed limit (80km/h)",
    6: "End of speed limit (80km/h)",
    7: "Speed limit (100km/h)",
    8: "Speed limit (120km/h)",
    9: "No passing",
    10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection",
    12: "Priority road",
    13: "Yield",
    14: "Stop",
    15: "No vehicles",
    16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry",
    18: "General caution",
    19: "Dangerous curve to the left",
    20: "Dangerous curve to the right",
    21: "Double curve",
    22: "Bumpy road",
    23: "Slippery road",
    24: "Road narrows on the right",
    25: "Road work",
    26: "Traffic signals",
    27: "Pedestrians",
    28: "Children crossing",
    29: "Bicycles crossing",
    30: "Beware of ice/snow",
    31: "Wild animals crossing",
    32: "End of all speed and passing limits",
    33: "Turn right ahead",
    34: "Turn left ahead",
    35: "Ahead only",
    36: "Go straight or right",
    37: "Go straight or left",
    38: "Keep right",
    39: "Keep left",
    40: "Roundabout mandatory",
    41: "End of no passing",
    42: "End of no passing by vehicles over 3.5 metric tons"
}

In [9]:
#define model 
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)), #32 filters, kernel 3x3, relu
    layers.MaxPooling2D((2, 2)), #max pooling layer
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(), #flatten to convert 3d feature maps to 1d feature vecotrs
    layers.Dense(128, activation='relu'), #fc layer w 128 units
    layers.Dense(43, activation='softmax') #o/p layer w 43 classes
])


C:\Users\aakaash\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 43)                  │           5,547 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 193,067 (754.17 KB)

 Trainable params: 193,067 (754.17 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
#compiling the model
#try with adam optimizer, SGD gave better results.
#optimizer- updates weights, loss fn- measures how well model predicts vs actual valeus
model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

"\nmodel.compile(optimizer='RMSprop',\n              loss=tf.keras.losses.SparseCategoricalCrossentropy(),\n              metrics=['accuracy'])\nmodel.compile(optimizer='RMSprop',\n              loss=tf.keras.losses.CategoricalCrossentropy(),\n              metrics=['accuracy'])"

In [12]:
train_images = np.array(train_images).astype('float32')
test_images = np.array(test_images).astype('float32') #convert img to np arrays
train_labels = np.array(train_labels) #labels are np arrays
test_labels = np.array(test_labels)

In [13]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels)) #fit the model

Epoch 1/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.5283 - loss: 1.7420 - val_accuracy: 0.7998 - val_loss: 0.8277
Epoch 2/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.8957 - loss: 0.3670 - val_accuracy: 0.8216 - val_loss: 0.7911
Epoch 3/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.9236 - loss: 0.2595 - val_accuracy: 0.8234 - val_loss: 0.9317
Epoch 4/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.9405 - loss: 0.2034 - val_accuracy: 0.8302 - val_loss: 0.9077
Epoch 5/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.9468 - loss: 0.1737 - val_accuracy: 0.8357 - val_loss: 1.0074
Epoch 6/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9529 - loss: 0.1578 - val_accuracy: 0.8394 - val_loss: 0.9662
Epoch 7/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9576 - loss: 0.1417 - val_accuracy: 0.8423 - val_loss: 1.0782
Epoch 8/10
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9573 -

In [18]:
model_1 = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu',padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu',padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu',padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(43, activation='softmax')
])
model_1.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)                    │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 8, 8, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 43)                  │           5,547 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 299,947 (1.14 MB)

 Trainable params: 299,947 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model_1.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
history = model_1.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels)) #fit the model

Epoch 1/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 33s 26ms/step - accuracy: 0.1891 - loss: 2.9291 - val_accuracy: 0.6297 - val_loss: 1.1944
Epoch 2/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 45s 29ms/step - accuracy: 0.6613 - loss: 1.0328 - val_accuracy: 0.7818 - val_loss: 0.8069
Epoch 3/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 41s 29ms/step - accuracy: 0.8234 - loss: 0.5636 - val_accuracy: 0.8086 - val_loss: 0.7473
Epoch 4/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 31s 25ms/step - accuracy: 0.8772 - loss: 0.4202 - val_accuracy: 0.8282 - val_loss: 0.7131
Epoch 5/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 46s 29ms/step - accuracy: 0.9045 - loss: 0.3278 - val_accuracy: 0.8323 - val_loss: 0.7094


In [22]:
model.compile(optimizer='RMSprop',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels)) #fit the model

Epoch 1/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.9717 - loss: 0.1013 - val_accuracy: 0.8429 - val_loss: 1.8406
Epoch 2/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.9737 - loss: 0.0921 - val_accuracy: 0.8432 - val_loss: 1.7814
Epoch 3/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.9751 - loss: 0.0901 - val_accuracy: 0.8416 - val_loss: 2.2632
Epoch 4/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.9764 - loss: 0.0850 - val_accuracy: 0.8409 - val_loss: 2.3331
Epoch 5/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.9736 - loss: 0.0990 - val_accuracy: 0.8367 - val_loss: 2.8320


In [26]:
train_labels_one_hot = to_categorical(train_labels, num_classes=43)
test_labels_one_hot = to_categorical(test_labels, num_classes=43)
model.compile(optimizer='RMSprop',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels_one_hot, epochs=5, validation_data=(test_images, test_labels_one_hot))


Epoch 1/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.9747 - loss: 0.0931 - val_accuracy: 0.8372 - val_loss: 2.7958
Epoch 2/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.9761 - loss: 0.0891 - val_accuracy: 0.8329 - val_loss: 3.0487
Epoch 3/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.9745 - loss: 0.0971 - val_accuracy: 0.8339 - val_loss: 3.2178
Epoch 4/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.9763 - loss: 0.0926 - val_accuracy: 0.8413 - val_loss: 3.4036
Epoch 5/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.9764 - loss: 0.0934 - val_accuracy: 0.8365 - val_loss: 3.6604


In [27]:
model.compile(optimizer='SGD',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels)) #fit the model

Epoch 1/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.9792 - loss: 0.0810 - val_accuracy: 0.8443 - val_loss: 3.5372
Epoch 2/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.9815 - loss: 0.0656 - val_accuracy: 0.8452 - val_loss: 3.4047
Epoch 3/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.9827 - loss: 0.0627 - val_accuracy: 0.8439 - val_loss: 3.5510
Epoch 4/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.9815 - loss: 0.0640 - val_accuracy: 0.8458 - val_loss: 3.6722
Epoch 5/5
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 25s 20ms/step - accuracy: 0.9823 - loss: 0.0602 - val_accuracy: 0.8464 - val_loss: 3.7555


In [28]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2) #evaluate it 

395/395 - 4s - 9ms/step - accuracy: 0.8464 - loss: 3.7555


In [29]:
print("\nTest accuracy:", test_acc)


Test accuracy: 0.8463974595069885
